--------------------------------
# Group: CHH
### Geoffrey Hung, Jiaqi Chen, Wendeng Hu
--------------------------------

In [46]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn import compose
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_log_error, median_absolute_error

import warnings
from joblib import dump, load
import time



# 1. Data processing
There is another notebook for EDA, we focus on modelling and evaluation part in this notebook. 

The data are from different data sources, the first step we need to do is to join the multiple data into one so we can train the model, the data size is big and we apply from tricks from Kaggle to reduce the dataset by changing the data type with less precision

Since the data is time series data, we cannot do random sampling to split the data for train-test set. Moreover, sklearn capability restricts us to include all the training data, for example, random forest will try to build n_estimator trees for each training, which is linearly proportional to running time, it becomes worse if we try to do cross validation

For our case, we include first 3 months as our training data and extra month as our testing data

--------------------------------

In [12]:
weather_df = pd.read_csv('ashrae-energy-prediction/cleaned_weather_final.csv')
building_df = pd.read_csv('ashrae-energy-prediction/building_metadata.csv')
train_df = pd.read_csv('ashrae-energy-prediction/train.csv')

In [13]:
# Data size reduction
# Original code from https://www.kaggle.com/gemartin/load-data-reduce-memory-usage

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

weather_df = reduce_mem_usage(weather_df)
building_df = reduce_mem_usage(building_df)
train_df = reduce_mem_usage(train_df)


Mem. usage decreased to  5.33 Mb (72.2% reduction)
Mem. usage decreased to  0.03 Mb (60.3% reduction)
Mem. usage decreased to 289.19 Mb (53.1% reduction)


## Merge the data & create train-test set
Below we try to join the data across 3 dataset.

First, we use train_df as the base dataset to join building metadata using building_id as the key.

Then we use the merged dataset to join weather data which is in site level by using site_id as the key.

In [14]:
# join different data together

tmp = train_df.merge(building_df, on='building_id', how='inner')
print(tmp.count())
print('')

tmp2 = tmp.merge(weather_df, on=['site_id', 'timestamp'], how='inner')
print(tmp2.count())
print('')

merged_data = tmp2[['building_id', 'meter', 'timestamp', 'meter_reading', 'site_id',
       'primary_use', 'square_feet', 'air_temperature', 'dew_temperature','wind_direction',
       'wind_speed', 'air_temperature_outliers_label','dew_temperature_outliers_label',
       'wind_direction_outliers_label', 'wind_speed_outliers_label']]

merged_data.head(3)

del tmp, tmp2

building_id      20216100
meter            20216100
timestamp        20216100
meter_reading    20216100
site_id          20216100
primary_use      20216100
square_feet      20216100
year_built        8088455
floor_count       3506933
dtype: int64

building_id                          20112649
meter                                20112649
timestamp                            20112649
meter_reading                        20112649
site_id                              20112649
primary_use                          20112649
square_feet                          20112649
year_built                            8007443
floor_count                           3493975
air_temperature                      20112649
cloud_coverage                       19636096
dew_temperature                      20112649
precip_depth_1_hr                    18466551
sea_level_pressure                   19333454
wind_direction                       20112649
wind_speed                           20112649
temp_rank       

## Feature engineering
Based on what we learned from EDA, it is time to apply transformation to the dataset (*some transformation are done to individual data already, this is transformation to the combined data)

Below we mainly focus on the time related data, we create features such as hour & weekday based on timestamp provided

We also apply log transformation to square feet to narrow the data range

Moreover, for first 4 months of the data, we observe there many abnormally zero in site 0, 13 and 15. Since our data only cover this 4 months, it is better for us to remove them from the data, otherwise our model will try to learn many zeros readings from the data

In [17]:
# Feature creation and remove abnormal sites

def transform(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['hour'] = np.uint8(df['timestamp'].dt.hour)
    df['day'] = np.uint8(df['timestamp'].dt.day)
    df['weekday'] = np.uint8(df['timestamp'].dt.weekday)
    df['month'] = np.uint8(df['timestamp'].dt.month)
    df['square_feet'] = np.log(df['square_feet'])
    return df

training_set = transform(training_set)
testing_set = transform(testing_set)

removed_site = [0, 13, 15]
training_set = training_set.loc[~training_set.site_id.isin(removed_site)]
testing_set = testing_set.loc[~testing_set.site_id.isin(removed_site)]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

## Train Test Split
Since we don't have labels for test set yet & sklearn is slow to process such amount of data, we restrict the data size by focusing on first 3 months as our training data (Jan - Mar) and an extra month as testing data (Apr), we do it in this order because **it is time series data, we cannot split the data by random sampling**


The training data contains ~4.7M records and testing data contains ~1.7M records

In [37]:
# Train Test Split

mask = (merged_data['timestamp'] > '2016-01-01') & (merged_data['timestamp'] <= '2016-03-31')
training_set = merged_data[mask]
print(training_set.shape)

mask = (merged_data['timestamp'] > '2016-04-01') & (merged_data['timestamp'] <= '2016-04-31')
testing_set = merged_data[mask]
print(testing_set.shape)

(4692951, 15)
(1682838, 15)


We split the data into features & targets, then we apply the log transformation to target column because the range is huge. It is better to modify the distribution and do the training first, then we can reverse the log transformation when we do the prediction

In [28]:
# Create features matrix and target column

X_train = training_set.drop(['timestamp', 'meter_reading'], axis=1)
y_train = training_set[['meter_reading']].values.ravel()

X_test = testing_set.drop(['timestamp', 'meter_reading'], axis=1)
y_test = testing_set[['meter_reading']].values.ravel()

y_train_log = np.log1p(y_train)

del training_set, testing_set

# --------------------------------
# 2. Pipeline & Modeling
This part we will create pipeline for the regression models and evaluate it

We apply standardization to numerical features for distance-aware models such as linear regression

Then we apply one hot encoding to building usage and expect different building usages will guide the model training
# --------------------------------

In [ ]:
def make_pipeline(regressor=None):
    "Create a single pipeline that processing the data and then fits the regressor." 

    # numeric features  
    numeric_features = ['square_feet', 'air_temperature', 'dew_temperature', 'wind_direction', 'wind_speed']
    numeric_transformer = Pipeline(steps=[
        ('scl', preprocessing.StandardScaler())
    ])
    
    # categorical features
    categorical_features = ['primary_use']
    categorical_transformer = Pipeline(steps=[
        ('onehot', preprocessing.OneHotEncoder(handle_unknown='ignore'))
    ])
    
    # combine
    preprocessor = compose.ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)]
        ,remainder='passthrough'
    )
    
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('regressor', regressor)])
    
    return pipeline

## Metrics we use:
### Root Mean Square Log Error (North Star Metric)
To summarize the difference b/w log(prediction) and log(actual), it is suitable for our case because meter readings range from single digit to tens of thousands

### Median Absolute Error
To interpret the result using the same unit (meter unit)

## Baseline: Linear Regression with L2 regularization

In [40]:
regressor = Ridge(normalize=False)

pipeline = make_pipeline(regressor)

pipeline.fit(X_train, y_train_log)

y_pred = np.expm1(pipeline.predict(X_test))
y_pred[y_pred < 0] = 0

medae_test = np.sqrt(mean_squared_log_error(y_test, y_pred))
print(f"{medae_test:.4f} RMSLE on test dataset")

medae_test = median_absolute_error(y_test, y_pred)
print(f"{medae_test:.4f} MedAE on test dataset")

1.6343 RMSLE on test dataset
44.7463 MedAE on test dataset


Our baseline model obtains 1.63 RMSLE & 44.75 MedAE

In our scenario, the prediction error is 44.75 meter units off in median

## Random Forest

In [51]:
# Random Forest

regressor = RandomForestRegressor(n_estimators=5,
                                  min_samples_leaf=200)

pipeline = make_pipeline(regressor)

pipeline.fit(X_train, y_train_log)

y_pred = np.expm1(pipeline.predict(X_test))
y_pred[y_pred < 0] = 0

medae_test = np.sqrt(mean_squared_log_error(y_test, y_pred))
print(f"{medae_test:.4f} RMSLE on test dataset")

medae_test = median_absolute_error(y_test, y_pred)
print(f"{medae_test:.4f} MedAE on test dataset")

0.9815 RMSLE on test dataset
11.8224 MedAE on test dataset


Random Forest without many tuning push the number down to 0.98 RMSLE and 11.82 MedAE which is a great improvement comparing with our baseline linear model

In our scenario, the prediction error is 11.82 meter units off in median, only quarter of the ridge regression

## ***!!!Time Consuming!!!*** Hyperparameter tuning for Random Forest ***!!!Time Consuming!!!***

### !!!Be careful if you really run it

This is very time consuming even for simple search space & limited iteration, **it takes literally ~5 hours to compute**, this suggests why we cannot use the whole data, even quarter of it can takes sklearn 5 hours to do cross validation for random forest.

This also implies for bigger dataset, we may need to use package other than sklearn such as lightgbm & XGBoost

In [47]:
warnings.filterwarnings("ignore")

time_start = time.time()

clf = RandomForestRegressor()

param_dist = {"n_estimators": [i for i in range(10, 50, 5)],
          "min_samples_leaf": [i for i in range(50, 200, 20)],
        }

regressor = RandomizedSearchCV(clf
                               ,param_distributions=param_dist
                               ,n_iter=5
                               ,cv=5
                               ,scoring="neg_mean_squared_log_error"
                              )

pipeline = make_pipeline(regressor)

pipeline.fit(X_train, y_train_log)

y_pred = np.expm1(pipeline.predict(X_test))
y_pred[y_pred < 0] = 0

medae_test = np.sqrt(mean_squared_log_error(y_test, y_pred))
print(f"{medae_test:.4f} RMSLE on test dataset")

medae_test = median_absolute_error(y_test, y_pred)
print(f"{medae_test:.4f} MedAE on test dataset")

time_end = time.time()

print(time_end - time_start)

dump(pipeline, 'pipeline.joblib')

0.9679 RMSLE on test dataset
11.3171 MedAE on test dataset
17515.99971985817


['pipeline.joblib']

Random Forest with hyperparameters tuning push the number down to 0.968 RMSLE and 11.32 MedAE which outperforms our previous random forest model.

In our scenario, the prediction error is 11.82 meter units off in median

## Conclusion:

In this project we have created a machine learning model using random forest to predict energy meter consumption, for the nature of time series data, we did sequential split of the data and keep the first three months as training data and an extra month as testing data.

We used linear model as our baseline followed by random forest. With hyperparameter tuning, we achieved 0.9679 RMSLE on test dataset (Kaggle leaderboard No.1 is around 0.93, although it is not directly compariable because of different dataset, this shows the order of magnitude of our solution is close to the mainstream). Median Absolute Error is only 11.32 meters units.

## Learnings:

From this project, it is a great opportunities for us to learn how to process the real world data.

-Data cleaning and EDA took more time than we expect and it is the most important to understand the problem

-Visualization is cruical to spot the patterns from data

-Data transformation is iteration, we need to explore and finetune all the time

-Model and evaluation is straightforward if we have solid pipeline
